In [1]:
from __future__ import print_function, division, absolute_import

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7fc320022208>

In [3]:
with open('./me_models/iBS.pickle', 'rb') as f:
    me = pickle.load(f)

In [4]:
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.lower_bound = 0
me.reactions.BIOMASS_BS_10_FWD_CPLX_dummy.upper_bound = 0
## Homogenize exchange reactions
for rxn in me.reactions:
    if 'EX_' in rxn.id:
        try:
            lb = bsub.reactions.get_by_id(rxn.id).lower_bound
            ub = bsub.reactions.get_by_id(rxn.id).upper_bound
            rxn.lower_bound = lb
            rxn.upper_bound = ub
        except:
            print(rxn.id, 'not in m_model')

        

EX_2ddglcn_e 0.0 999999.0
EX_2hxmp_e 0.0 999999.0
EX_2pg_e 0.0 999999.0
EX_2pglyc_e 0.0 999999.0
EX_3amba_e 0.0 999999.0
EX_3amp_e 0.0 999999.0
EX_3cmp_e 0.0 999999.0
EX_3gmp_e 0.0 999999.0
EX_3pg_e 0.0 999999.0
EX_3ump_e 0.0 999999.0
EX_4abut_e 0.0 999999.0
EX_5mtr_e 0.0 999999.0
EX_6pgc_e 0.0 999999.0
EX_Larab_e 0.0 999999.0
EX_Lcyst_e 0.0 999999.0
EX_abt__L_e 0.0 999999.0
EX_ac_e 0.0 999999.0
EX_acac_e 0.0 999999.0
EX_acgam_e 0.0 999999.0
EX_acmana_e 0.0 999999.0
EX_acnam_e 0.0 999999.0
EX_actn__R_e 0.0 999999.0
EX_ade_e 0.0 999999.0
EX_adn_e 0.0 999999.0
EX_akg_e 0.0 999999.0
EX_ala_B_e 0.0 999999.0
EX_ala__D_e 0.0 999999.0
EX_ala_L_Thr__L_e 0.0 999999.0
EX_ala_L_asp__L_e 0.0 999999.0
EX_ala__L_e 0.0 999999.0
EX_ala_L_gln__L_e 0.0 999999.0
EX_ala_L_glu__L_e 0.0 999999.0
EX_L_alagly_e 0.0 999999.0
EX_ala_L_his__L_e 0.0 999999.0
EX_ala_L_leu__L_e 0.0 999999.0
EX_alaala_e 0.0 999999.0
EX_alltn_e 0.0 999999.0
EX_amp_e 0.0 999999.0
EX_amylase_e 0.0 999999.0
EX_antim_e 0.0 999999.0
EX_ar

In [5]:
solve_me_model(me, 1., min_mu = .1, precision=1e-2, using_soplex=False)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 89.798749 seconds
Finished substituting S,lb,ub in 3.254184 seconds
Finished makeME_LP in 0.667203 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.5 0.5 1
Finished substituting S,lb,ub in 3.199147 seconds
Finished makeME_LP in 0.656673 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 3.194189 seconds
Finished makeME_LP in 0.659029 seconds
Getting MINOS parameters from ME_NLP...
3 0.0 0.0 0.125 0.125 1
Finished substituting S,lb,ub in 3.252431 seconds
Finished makeME_LP in 0.668978 seconds
Getting MINOS parameters from ME_NLP...
4 0.0 0.0 0.0625 0.0625 1
Finished substituting S,lb,ub in 3.197753 seconds
Finished makeME_LP in 0.664434 seconds
Getting MINOS parameters from ME_NLP...
5 0.0 0.0 0.03125 0.03125 1
Finished substituting S,lb,ub in 3.191941 seconds
Finished makeME_LP in 0.656022 seconds
Getting MINOS parameters from ME_NLP...
6 0.0 0.

AttributeError: 'NoneType' object has no attribute 'x_dict'

In [ ]:
with open('./me_models/solution.pickle', 'wb') as solution:
    pickle.dump(me,solution)

In [ ]:
filename = 'solution_summary.csv'
file = open(filename,'w')
file.write('ID, reaction, LB, UB, V \n')
for rxn in me.reactions:
    try:
        reaction = rxn.reaction
    except:
        reaction = 'NA'
    string = rxn.id + ',' + reaction + ',' + str(rxn.lower_bound) + ',' + str(rxn.upper_bound) +\
    ',' +  str(me.solution.x_dict[rxn.id]) + '\n'
    file.write(string)
file.close()

In [ ]:
# Display metabolic flux on escher map
show_escher_map(me).display_in_notebook()

In [ ]:
flux_dict = me.get_metabolic_flux()

In [ ]:
filename = 'fluxdist_bsub.csv'
file = open(filename,'w')
for rxn in flux_dict.keys():
    flux = flux_dict[rxn]
    file.write(rxn + ',' + str(flux) + '\n')
file.close()

In [ ]:
filename = 'fluxdist_m_bsub.csv'
file = open(filename,'w')
for rxn in bsub.reactions:
    flux = bsub.solution.x_dict[rxn.id]
    file.write(rxn.id + ',' + str(flux) + '\n')
file.close()